In [4]:
sc

In [5]:
import csv

In [28]:
data =  sc.textFile('coalesce_joined_weather_collisions_sorted_fixed.csv')
df = spark.read.load('coalesce_joined_weather_collisions_sorted_fixed.csv', format='csv', header=True, inferSchema=False)


df.registerTempTable('df')

In [14]:
df.take(2)

[Row(BOROUGH=u'QUEENS', CONTRIBUTING_FACTOR_VEHICLE_1=u'Unspecified', CONTRIBUTING_FACTOR_VEHICLE_2=u'Unspecified', CONTRIBUTING_FACTOR_VEHICLE_3=None, CONTRIBUTING_FACTOR_VEHICLE_4=None, CONTRIBUTING_FACTOR_VEHICLE_5=None, CROSS_STREET_NAME=u'70 DRIVE', DATETIME=datetime.datetime(2016, 6, 20, 21, 6), LATITUDE=None, LOCATION=None, LONGITUDE=None, NUMBER_OF_CYCLIST_INJURED=0, NUMBER_OF_CYCLIST_KILLED=0, NUMBER_OF_MOTORIST_INJURED=0, NUMBER_OF_MOTORIST_KILLED=0, NUMBER_OF_PEDESTRIANS_INJURED=0, NUMBER_OF_PEDESTRIANS_KILLED=0, NUMBER_OF_PERSONS_INJURED=0, NUMBER_OF_PERSONS_KILLED=0, OFF_STREET_NAME=None, ON_STREET_NAME=u'METROPOLITAN AVENUE', UNIQUE_KEY=3465402, VEHICLE_TYPE_CODE_1=u'PASSENGER VEHICLE', VEHICLE_TYPE_CODE_2=u'PASSENGER VEHICLE', VEHICLE_TYPE_CODE_3=None, VEHICLE_TYPE_CODE_4=None, VEHICLE_TYPE_CODE_5=None, ZIP_CODE=11375, _c28=58671, Conditions=u'Clear', DateUTC=datetime.datetime(2016, 6, 21, 1, 51), DatetimeEDT=datetime.datetime(2016, 6, 20, 21, 51), Dew_PointF=63.0, Event

In [29]:
df.dtypes

[('BOROUGH', 'string'),
 ('CONTRIBUTING_FACTOR_VEHICLE_1', 'string'),
 ('CONTRIBUTING_FACTOR_VEHICLE_2', 'string'),
 ('CONTRIBUTING_FACTOR_VEHICLE_3', 'string'),
 ('CONTRIBUTING_FACTOR_VEHICLE_4', 'string'),
 ('CONTRIBUTING_FACTOR_VEHICLE_5', 'string'),
 ('CROSS_STREET_NAME', 'string'),
 ('DATETIME', 'string'),
 ('LATITUDE', 'string'),
 ('LOCATION', 'string'),
 ('LONGITUDE', 'string'),
 ('NUMBER_OF_CYCLIST_INJURED', 'string'),
 ('NUMBER_OF_CYCLIST_KILLED', 'string'),
 ('NUMBER_OF_MOTORIST_INJURED', 'string'),
 ('NUMBER_OF_MOTORIST_KILLED', 'string'),
 ('NUMBER_OF_PEDESTRIANS_INJURED', 'string'),
 ('NUMBER_OF_PEDESTRIANS_KILLED', 'string'),
 ('NUMBER_OF_PERSONS_INJURED', 'string'),
 ('NUMBER_OF_PERSONS_KILLED', 'string'),
 ('OFF_STREET_NAME', 'string'),
 ('ON_STREET_NAME', 'string'),
 ('UNIQUE_KEY', 'string'),
 ('VEHICLE_TYPE_CODE_1', 'string'),
 ('VEHICLE_TYPE_CODE_2', 'string'),
 ('VEHICLE_TYPE_CODE_3', 'string'),
 ('VEHICLE_TYPE_CODE_4', 'string'),
 ('VEHICLE_TYPE_CODE_5', 'string'),

In [21]:
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
def filter_general(x):
    if x > 65:
        return float(x)
    else:
        return float(0)
    
sparkUDF = udf(filter_general, FloatType())
newcol = df.withColumn("Dew_PointF", sparkUDF("Dew_PointF"))
newcol.take(5)


[Row(BOROUGH=u'QUEENS', CONTRIBUTING_FACTOR_VEHICLE_1=u'Unspecified', CONTRIBUTING_FACTOR_VEHICLE_2=u'Unspecified', CONTRIBUTING_FACTOR_VEHICLE_3=None, CONTRIBUTING_FACTOR_VEHICLE_4=None, CONTRIBUTING_FACTOR_VEHICLE_5=None, CROSS_STREET_NAME=u'70 DRIVE', DATETIME=datetime.datetime(2016, 6, 20, 21, 6), LATITUDE=None, LOCATION=None, LONGITUDE=None, NUMBER_OF_CYCLIST_INJURED=0, NUMBER_OF_CYCLIST_KILLED=0, NUMBER_OF_MOTORIST_INJURED=0, NUMBER_OF_MOTORIST_KILLED=0, NUMBER_OF_PEDESTRIANS_INJURED=0, NUMBER_OF_PEDESTRIANS_KILLED=0, NUMBER_OF_PERSONS_INJURED=0, NUMBER_OF_PERSONS_KILLED=0, OFF_STREET_NAME=None, ON_STREET_NAME=u'METROPOLITAN AVENUE', UNIQUE_KEY=3465402, VEHICLE_TYPE_CODE_1=u'PASSENGER VEHICLE', VEHICLE_TYPE_CODE_2=u'PASSENGER VEHICLE', VEHICLE_TYPE_CODE_3=None, VEHICLE_TYPE_CODE_4=None, VEHICLE_TYPE_CODE_5=None, ZIP_CODE=11375, _c28=58671, Conditions=u'Clear', DateUTC=datetime.datetime(2016, 6, 21, 1, 51), DatetimeEDT=datetime.datetime(2016, 6, 20, 21, 51), Dew_PointF=0.0, Events

In [33]:
a = sqlContext.sql('select * from df where Rain=1')
a.coalesce(1).write.csv('rain', header=True)

a = sqlContext.sql('select * from df where Snow=1')
a.coalesce(1).write.csv('snow', header=True)

a = sqlContext.sql('select * from df where Freezing=1')
a.coalesce(1).write.csv('freezing', header=True)

a = sqlContext.sql('select * from df where LowVisibility=1')
a.coalesce(1).write.csv('foggy', header=True)

a = sqlContext.sql('select * from df where Windy=1')
a.coalesce(1).write.csv('windy', header=True)


df = df.filter(df.LATITUDE != '0')

from pyspark.sql.functions import col


a = df.where(col("LOCATION").isNotNull())
a.coalesce(1).write.csv('geodata', header=True)

In [25]:
test.count()

104796